In [1]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVR, SVC
from sklearn.linear_model import LogisticRegression, ElasticNet
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report, r2_score
from xgboost import XGBRegressor, XGBClassifier
import spacy
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier, XGBRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegression, ElasticNet
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from matplotlib import pyplot as plt
from lightgbm import LGBMRegressor, LGBMClassifier  # Added LightGBM imports
from itertools import cycle
from sklearn.metrics import auc, confusion_matrix, f1_score, precision_score, recall_score, roc_curve
from sklearn.model_selection import learning_curve
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import RocCurveDisplay



In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import spacy
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from datetime import datetime, timedelta
import requests

class TrelloIntegrationManager:
    def __init__(self, api_key, token):
        self.api_key = api_key
        self.token = token
        self.base_url = 'https://api.trello.com/1'

    def _make_request(self, method, endpoint, params=None, data=None):
        """Generic method to make API requests"""
        url = f'{self.base_url}/{endpoint}'
        
        # Always include key and token in params
        request_params = {
            'key': self.api_key,
            'token': self.token
        }
        
        if params:
            request_params.update(params)
        
        try:
            if method == 'GET':
                response = requests.get(url, params=request_params)
            elif method == 'POST':
                response = requests.post(url, params=request_params, json=data)
            elif method == 'PUT':
                response = requests.put(url, params=request_params, json=data)
            else:
                raise ValueError(f"Unsupported HTTP method: {method}")
            
            response.raise_for_status()
            return response.json()
        
        except requests.exceptions.RequestException as e:
            print(f"Trello API Request Error: {e}")
            return None

    def create_card(self, list_id, name, description=None, due_date=None):
        """Create a new card in a specific list"""
        endpoint = 'cards'
        data = {
            'idList': str(list_id),
            'name': str(name),
            'desc': str(description or '')
        }
        
        if due_date:
            # Ensure due_date is a string in ISO format
            if isinstance(due_date, (datetime)):
                due_date = due_date.isoformat()
            data['due'] = str(due_date)
        
        return self._make_request('POST', endpoint, data=data)

    def update_card_due_date(self, card_id, due_date):
        """Update a card's due date"""
        endpoint = f'cards/{card_id}'
        data = {'due': due_date}
        return self._make_request('PUT', endpoint, data=data)

    def get_boards(self):
        """Retrieve all boards for the authenticated user"""
        endpoint = 'members/me/boards'
        params = {
            'fields': 'name,url,id',
            'lists': 'open'
        }
        return self._make_request('GET', endpoint, params)

    def get_lists(self, board_id):
        """Get lists for a specific board"""
        endpoint = f'boards/{board_id}/lists'
        params = {
            'fields': 'name,id'
        }
        return self._make_request('GET', endpoint, params)

class PriorityBasedTaskManager:
    def __init__(self):
        # Priority mapping (Low:0, Medium:1, High:2)
        self.priority_mapping = {'Low': 2, 'Medium': 1, 'High': 0}
        self.priority_reverse_mapping = {2: 'Low', 1: 'Medium', 0: 'High'}
        self.priority_encoder = LabelEncoder()
        self.model = None
        self.nlp_processor = NLPProcessor()
        
    def preprocess_data(self, data):
        """Prepare data for modeling focusing only on priority"""
        features = data.copy()
        
        # Ensure priority is categorical
        features['Priority'] = features['Priority'].astype(str)
        
        # Encode priority if not already done
        if not hasattr(self.priority_encoder, 'classes_'):
            features['priority_encoded'] = self.priority_encoder.fit_transform(features['Priority'])
        else:
            # Handle new priority values not seen during training
            features['Priority'] = features['Priority'].apply(
                lambda x: 'Medium' if x not in self.priority_encoder.classes_ else x
            )
            features['priority_encoded'] = self.priority_encoder.transform(features['Priority'])
        
        return features
    
    def train_model(self, data):
        """Train a priority prediction model"""
        prepared_data = self.preprocess_data(data)
        
        # Use NLP to extract features from task description
        text_features = self.nlp_processor.extract_text_features(data['Description'])
        
        # Combine features
        X = pd.concat([
            pd.DataFrame(text_features),
            prepared_data[['priority_encoded']]  # Current priority as feature
        ], axis=1)
        
        # Convert all column names to strings to avoid the TypeError
        X.columns = X.columns.astype(str)
        
        y = prepared_data['priority_encoded']
        
        # Print class distribution before splitting
        print("Class distribution in training data:")
        print(pd.Series(y).value_counts())
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )
        
        # Simple model focusing on priority
        self.model = RandomForestClassifier(
            n_estimators=100,
            max_depth=5,
            class_weight='balanced',
            random_state=42
        )
        
        self.model.fit(X_train, y_train)
        
        # Evaluate
        y_pred = self.model.predict(X_test)
        print("Model Accuracy:", accuracy_score(y_test, y_pred))
        print("\nClassification Report:")
        print(classification_report(y_test, y_pred))
        
        # Add prediction distribution check
        print("Prediction distribution:")
        print(pd.Series(y_pred).value_counts())

        return self.model
        
    def predict_priority(self, task_description, current_priority='Medium'):
        """Predict task priority based on description and current priority"""
        if self.model is None:
            raise ValueError("Model must be trained before making predictions")
            
        # Process text features
        text_features = self.nlp_processor.extract_text_features([task_description])
        
        # Prepare input data
        input_data = pd.DataFrame(text_features)
        input_data['priority_encoded'] = self.priority_mapping.get(current_priority, 1)  # Default to Medium
        
        # Convert all column names to strings to match training data
        input_data.columns = input_data.columns.astype(str)
        
        # Predict
        priority_num = self.model.predict(input_data)[0]
        return self.priority_reverse_mapping[priority_num]
    
    def categorize_task(self, description):
        """Categorize task using NLP"""
        return self.nlp_processor.categorize_task(description)

class NLPProcessor:
    def __init__(self):
        self.nlp = spacy.load("en_core_web_sm")
        self.text_vectorizer = TfidfVectorizer(max_features=100)
        
    def extract_text_features(self, descriptions):
        """Extract NLP-based features from task descriptions"""
        # Process descriptions with spaCy
        processed_texts = [" ".join([token.lemma_ for token in self.nlp(doc)]) for doc in descriptions]
        
        # Vectorize text
        if not hasattr(self.text_vectorizer, 'vocabulary_'):
            text_features = self.text_vectorizer.fit_transform(processed_texts)
        else:
            text_features = self.text_vectorizer.transform(processed_texts)
            
        return pd.DataFrame.sparse.from_spmatrix(text_features)
    
    def categorize_task(self, description):
        """Categorize task based on description content"""
        doc = self.nlp(description)
        categories = set()
        
        # Entity-based categorization
        for ent in doc.ents:
            if ent.label_ == "ORG":
                categories.add("Organization")
            elif ent.label_ == "PERSON":
                categories.add("Person-Related")
            elif ent.label_ == "DATE":
                categories.add("Time-Sensitive")
            elif ent.label_ == "MONEY":
                categories.add("Financial")
        
        # Verb-based categorization
        if any(token.lemma_ in ["report", "analyze", "research"] for token in doc):
            categories.add("Analytical")
            
        if any(token.lemma_ in ["meeting", "call", "discuss"] for token in doc):
            categories.add("Communication")
            
        if any(token.lemma_ in ["buy", "purchase", "order"] for token in doc):
            categories.add("Procurement")
            
        if any(token.lemma_ in ["write", "document", "create"] for token in doc):
            categories.add("Content-Creation")
            
        return list(categories) if categories else ["General"]

# Main execution
if __name__ == "__main__":
    # Trello API credentials
    TRELLO_API_KEY = '9bd35201acaf3e9f629af9c7648b6e26'
    TRELLO_TOKEN = 'ATTAd2ea9551043c1590ec93062a5df503dfa2b5c5b483fdf18b93c8ab176b981499380E20EE'

    # Load sample data
    print("Loading datasets...")
    user_tasks = pd.read_csv("modified_user_tasks.csv")
    ml_training_data = user_tasks[['Description', 'Priority']].copy()

    # Initialize and train the system
    task_manager = PriorityBasedTaskManager()
    task_manager.train_model(ml_training_data)

    # Initialize Trello integration
    trello_manager = TrelloIntegrationManager(TRELLO_API_KEY, TRELLO_TOKEN)

    # Get the first board (or create one if none exists)
    boards = trello_manager.get_boards()
    if not boards:
        print("No boards found, please create one in Trello first")
        exit()
        
    sample_board = boards[0]
    print(f"Using board: {sample_board['name']}")

    # Get lists from the board
    lists = trello_manager.get_lists(sample_board['id'])
    if not lists:
        print("No lists found in the board")
        exit()



Loading datasets...
Class distribution in training data:
priority_encoded
0    1379
2    1322
1    1299
Name: count, dtype: int64


C:\Users\youss\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\youss\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


Model Accuracy: 1.0

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       276
           1       1.00      1.00      1.00       260
           2       1.00      1.00      1.00       264

    accuracy                           1.00       800
   macro avg       1.00      1.00      1.00       800
weighted avg       1.00      1.00      1.00       800

Prediction distribution:
0    276
2    264
1    260
Name: count, dtype: int64
Using board: My Trello board


C:\Users\youss\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(



Processing task: Conduct team meeting to discuss next sprint goals
Description: Schedule and prepare agenda for sprint planning
Predicted Priority: Medium
Categories: ['General']
Created Trello card with ID: 6803ec274499460e33120b53
Set Medium priority reminder for card 6803ec274499460e33120b53 due at 2025-04-20T20:32:07.659885

Processing task: Fix critical bug in payment processing
Description: Customers reporting payment failures in checkout
Predicted Priority: Medium
Categories: ['Analytical']


C:\Users\youss\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


Created Trello card with ID: 6803ec28e7f040e22e668940
Set Medium priority reminder for card 6803ec28e7f040e22e668940 due at 2025-04-20T20:32:08.436147

Processing task: Update documentation for new API endpoints
Description: Document the 3 new endpoints added last week
Predicted Priority: Medium
Categories: ['Content-Creation', 'Time-Sensitive']


C:\Users\youss\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


Created Trello card with ID: 6803ec29c940cc6b7349a435
Set Medium priority reminder for card 6803ec29c940cc6b7349a435 due at 2025-04-20T20:32:09.223689


In [20]:
# Sample tasks to test with actual priorities
sample_tasks = [
    {"name": "Resolve server outage affecting customer access", 
     "desc": "Urgent server issue preventing customers from accessing services",
     "actual_priority": "High"},
    
    {"name": "Conduct team meeting to discuss next sprint goals", 
     "desc": "Schedule and prepare agenda for sprint planning",
     "actual_priority": "Medium"},
    
    {"name": "Reply to non-urgent emails and notifications", 
     "desc": "Respond to general inquiries and notifications",
     "actual_priority": "Low"}
]

# Add sample tasks to Trello and set reminders
for task in sample_tasks:
    # Predict priority
    predicted_priority = task_manager.predict_priority(task['desc'])
    categories = task_manager.categorize_task(task['desc'])
    
    print(f"\nProcessing task: {task['name']}")
    print(f"Description: {task['desc']}")
    print(f"Actual Priority: {task['actual_priority']}")
    print(f"Predicted Priority: {predicted_priority}")
    print(f"Categories: {categories}")
    
    # Create card in Trello (using first list for simplicity)
    card = trello_manager.create_card(
        list_id=lists[0]['id'],
        name=task['name'],
        description=f"{task['desc']}\n\nActual Priority: {task['actual_priority']}\nPredicted Priority: {predicted_priority}",
        due_date=None  # Will be set based on priority
    )
    
    if card:
        print(f"Created Trello card with ID: {card['id']}")
        set_trello_reminders(card['id'], predicted_priority)
    else:
        print("Failed to create Trello card")

C:\Users\youss\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(



Processing task: Resolve server outage affecting customer access
Description: Urgent server issue preventing customers from accessing services
Actual Priority: High
Predicted Priority: High
Categories: ['General']
Created Trello card with ID: 6803ed165531c1aaa1500471
Set High priority reminder for card 6803ed165531c1aaa1500471 due at 2025-04-19T22:36:07.064552

Processing task: Conduct team meeting to discuss next sprint goals
Description: Schedule and prepare agenda for sprint planning
Actual Priority: Medium
Predicted Priority: Medium
Categories: ['General']


C:\Users\youss\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


Created Trello card with ID: 6803ed170a7ae679867db21c
Set Medium priority reminder for card 6803ed170a7ae679867db21c due at 2025-04-20T20:36:07.865838

Processing task: Reply to non-urgent emails and notifications
Description: Respond to general inquiries and notifications
Actual Priority: Low
Predicted Priority: Medium
Categories: ['General']


C:\Users\youss\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


Created Trello card with ID: 6803ed187162f86c9cee8fb6
Set Medium priority reminder for card 6803ed187162f86c9cee8fb6 due at 2025-04-20T20:36:08.636211
